# 2 Big Earth Data Access
### from different data repositories, e.g. Climate Data Store, Google Earth Engine or Earth on AWS

<hr>
<a href="./01_Introduction_Jupyter_Notebooks.ipynb"><< 1 - Introduction to Jupyter Notebooks</a>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space> <a href="./03_Introduction_Jupyter_widgets.ipynb">3 - Introducing Jupyter widgets >></a>

## Overview of Big Earth Data repositories

There is a lot of open environmental data available. The problem is often that people do not know where to find the data and how to access it.
We will go through three Big Earth Data repositories today.

* [Copernicus Climate Data Store (CDS)](https://cds.climate.copernicus.eu/cdsapp#!/home)
* [Google Earth Engine (GEE)](https://earthengine.google.com/)
* [Earth on Amazon Web Services (AWS)](https://aws.amazon.com/earth/)



## 2.1 Copernicus Climate Data Store (CDS)

Copernicus Climate Data Store (CDS) consists of two parts:
* Access to Climate Datasets via a [web interface](https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset) or programmatically via the [Climate Data Store API](https://cds.climate.copernicus.eu/api-how-to)
* Analyse and visualise climate data with the [Climate Data Store toolbox (Python interface)](https://cds.climate.copernicus.eu/user/login?destination=/toolbox-user)
* Data are available in NetCDF or GRIB

### Example to download ERA5 data in .grib and open it with xarray

<hr>

## 2.2 Google Earth Engine

Google Earth Engine (GEE) 
* [More info on GEE Python API](https://developers.google.com/earth-engine/python_install_manual)

In [4]:
import ee

In [208]:
%matplotlib inline

In [269]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, FullScreenControl, Marker
import ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output


ee.Initialize()

### Load collection and retrieve projection information

In [245]:
collection = ee.ImageCollection('projects/ecmwf/era5_monthly')

collection_img = collection.limit(1).first()
collection_img_proj = coll_img.select(0).projection()

# See information about collection
collection.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'version': 1552687984494899,
 'id': 'projects/ecmwf/era5_monthly',
 'features': [{'type': 'Image',
   'bands': [{'id': 'tp',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'dimensions': [1440, 721],
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0.0, -180.125, 0.0, -0.25, 90.125]}],
   'version': 1552685691902161,
   'id': 'projects/ecmwf/era5_monthly/197901_tp',
   'properties': {'system:time_start': 283996800000.0,
    'month': 1.0,
    'year': 1979.0,
    'system:footprint': {'type': 'LinearRing',
     'coordinates': [[-180.0, -90.0],
      [180.0, -90.0],
      [180.0, 90.0],
      [-180.0, 90.0],
      [-180.0, -90.0]]},
    'system:time_end': 286675200000.0,
    'system:asset_size': 4837680,
    'system:index': '197901_tp'}},
  {'type': 'Image',
   'bands': [{'id': 'tp',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'dimensions': [1440, 721],
     'crs': 'EPSG:4326',
     'crs_transform':

### Process mean precipitation for each month based on entire time series

In [48]:
months = range(1,13)

# Store images in a list
img_list = []
for i in months:
    collection_filtered = collection.filter(ee.Filter.calendarRange(i,i, 'month'))
    collection_red = collection_filtered.reduce(ee.Reducer.mean())
    # if reducer function is applied to an image collection, the output does not have any projection information, as collection can contain \ 
    # images with different projection information. Thus, one can set the projection to each image
    collection_red_proj = collection_red.setDefaultProjection(collection_img_proj)
    img_list.append(collection_red_proj)
    
img_list

### Build an Image Collection of the resulting image list

In [249]:
meanMonths_collection = ee.ImageCollection.fromImages(img_list)
meanMonths_collection.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'tp_mean',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0.0, -180.125, 0.0, -0.25, 90.125]}],
   'properties': {'system:index': '0'}},
  {'type': 'Image',
   'bands': [{'id': 'tp_mean',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0.0, -180.125, 0.0, -0.25, 90.125]}],
   'properties': {'system:index': '1'}},
  {'type': 'Image',
   'bands': [{'id': 'tp_mean',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0.0, -180.125, 0.0, -0.25, 90.125]}],
   'properties': {'system:index': '2'}},
  {'type': 'Image',
   'bands': [{'id': 'tp_mean',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0.0, -180.125, 0.0, -0.25, 90.125]}],

In [248]:
img = GetTileLayerUrl(meanMonths_collection.first())
img

'https://earthengine.googleapis.com/map/f2aa0163059249154904535c506a0f4f/{z}/{x}/{y}?token=29c35c8501fe9f7593c39ad6f83ca5d4'

In [278]:
map1 = ipyleaflet.Map(
    zoom=2,
    layout={'height':'500px'},
)

map1.add_layer(ipyleaflet.TileLayer(url=img))
control = FullScreenControl()
map1.add_control(control)

map1

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [254]:
tp_point = meanMonths_collection.getRegion(point,500).getInfo()
point = ee.Geometry.Point(lon,lat)
header = tp_point[0]

data = tp_point[1:]
ydata = [row[4]*1000 for row in data]

tp = go.Bar(
    x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
    y=ydata,
    name='Total precipitation in mm',
    marker=dict(
        color='rgb(204,204,204)',
    ))


data = [tp]
layout = go.Layout(
    title='Total precipitation in mm',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='test')



<hr>

## 2.3 Earth on AWS

<hr>
&copy; 2019 | Julia Wagemann
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>